In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

# Indexing

In [3]:
dates=pd.date_range('1/2/2021', periods=8)
df = pd.DataFrame(np.random.randn(8, 4), columns=list("ABCD"),
                 index=dates)
df

,A,B,C,D
2021-01-02,1.629301,-0.009352,-0.905488,0.165572
2021-01-03,0.244092,1.313733,-0.347823,1.123616
2021-01-04,2.190245,-0.021858,1.136657,0.275776
2021-01-05,-0.676463,1.244879,0.791442,-0.511738
2021-01-06,-0.405205,0.502825,-0.922520,-1.285305
2021-01-07,-1.373553,0.056632,-0.666912,0.813599
2021-01-08,0.203159,1.045986,2.250007,-0.773191
2021-01-09,-1.254659,0.956169,-1.418185,-2.475736


In [8]:
s = df['A']
s[dates[5]]

-1.3735534007078942

In [10]:
df[['B', 'A']] = df[['A', 'B']]
df

,A,B,C,D
2021-01-02,-0.009352,1.629301,-0.905488,0.165572
2021-01-03,1.313733,0.244092,-0.347823,1.123616
2021-01-04,-0.021858,2.190245,1.136657,0.275776
2021-01-05,1.244879,-0.676463,0.791442,-0.511738
2021-01-06,0.502825,-0.405205,-0.922520,-1.285305
2021-01-07,0.056632,-1.373553,-0.666912,0.813599
2021-01-08,1.045986,0.203159,2.250007,-0.773191
2021-01-09,0.956169,-1.254659,-1.418185,-2.475736


In [18]:
df.loc[:, ['B', 'A']]= df[['A', 'B']].to_numpy()
df

,A,B,C,D
2021-01-02,-0.009352,1.629301,-0.905488,0.165572
2021-01-03,1.313733,0.244092,-0.347823,1.123616
2021-01-04,-0.021858,2.190245,1.136657,0.275776
2021-01-05,1.244879,-0.676463,0.791442,-0.511738
2021-01-06,0.502825,-0.405205,-0.922520,-1.285305
2021-01-07,0.056632,-1.373553,-0.666912,0.813599
2021-01-08,1.045986,0.203159,2.250007,-0.773191
2021-01-09,0.956169,-1.254659,-1.418185,-2.475736


In [30]:
sa = pd.Series([1, 2, 3], index=list('abc'))
sa.b
s['2021-01-07']

0.05663161423750495

In [31]:
dfa = df.copy()
dfa.A

2021-01-02   -0.009352
2021-01-03    1.313733
2021-01-04   -0.021858
2021-01-05    1.244879
2021-01-06    0.502825
2021-01-07    0.056632
2021-01-08    1.045986
2021-01-09    0.956169
Freq: D, Name: A, dtype: float64

In [32]:
print(sa)
sa.a = 5
sa

a    1
b    2
c    3
dtype: int64


a    5
b    2
c    3
dtype: int64

In [37]:
dfa.A = list(range(len(dfa.index))) # # ok if A already exists
dfa

dfa.E = list(range(len(dfa.index))) ## Error - UserWarning: Pandas doesn't allow columns to be created via a new attribute name
dfa

# use below form to create a new ˓→column
dfa['E'] = list(range(len(dfa.index)))
dfa

,A,B,C,D,E
2021-01-02,0,1.629301,-0.905488,0.165572,0
2021-01-03,1,0.244092,-0.347823,1.123616,1
2021-01-04,2,2.190245,1.136657,0.275776,2
2021-01-05,3,-0.676463,0.791442,-0.511738,3
2021-01-06,4,-0.405205,-0.922520,-1.285305,4
2021-01-07,5,-1.373553,-0.666912,0.813599,5
2021-01-08,6,0.203159,2.250007,-0.773191,6
2021-01-09,7,-1.254659,-1.418185,-2.475736,7


In [41]:
x = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]})
x.iloc[1] = {'x':9, 'y':99}
x

,x,y
0,1,3
1,9,99
2,3,5


In [39]:
df = pd.DataFrame({'one': [1., 2., 3.]})
df.two = [4, 5, 6] ## Error - UserWarning: Pandas doesn't allow columns to be created via a new attribute name
df

<ipython-input-39-7e0e54ff9d76>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.two = [4, 5, 6] ## Error - UserWarning: Pandas doesn't allow columns to be created via a new attribute name


,one
0,1.0
1,2.0
2,3.0


In [49]:
s[:5]
s[::2]
s[::-1]

#Note that setting works as well:
s2 = s.copy()
s2[:5] = 0
s2

2021-01-02    0.000000
2021-01-03    0.000000
2021-01-04    0.000000
2021-01-05    0.000000
2021-01-06    0.000000
2021-01-07    0.056632
2021-01-08    1.045986
2021-01-09    0.956169
Freq: D, Name: A, dtype: float64

In [51]:
df[:3]
df[::-1]

,one
2,3.0
1,2.0
0,1.0


In [61]:
df1 = pd.DataFrame(np.random.randn(5, 4), columns=list('ABCD'), index=pd.date_range('20130101', periods=5))
#df1.loc[2:4] #TypeError: cannot do slice indexing on DatetimeIndex with these indexers [2] of type int
df1.iloc[2:4]
df1.loc[[True, False, True, False, False]]

,A,B,C,D
2013-01-01,-0.264241,-0.412795,0.002391,-1.200910
2013-01-03,-0.325368,-0.292521,0.554763,-0.287031


In [66]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))
s1.loc['c':]
s1.loc['b']
s1.loc['c':] = 0
s1

a   -0.243911
b   -0.918299
c    0.000000
d    0.000000
e    0.000000
f    0.000000
dtype: float64

In [71]:
df1 = pd.DataFrame(np.random.randn(6, 4), index=list('abcdef'), columns=list('ABCD'))
df1.loc[['a', 'b', 'd'], :]
df1.loc['a']
df1.loc[:, df1.loc['a'] > 0]

,C
a,1.021648
b,-0.074328
c,0.903928
d,-1.053766
e,-0.091051
f,-2.184279


In [74]:
mask = pd.array([True, False, True, False, pd.NA, False], dtype="boolean")
mask
df1[mask]

,A,B,C,D
a,-0.072105,-0.904690,1.021648,-0.573483
c,-0.603085,1.747245,0.903928,0.345961


In [111]:
dft = pd.DataFrame(np.random.randn(1000000, 6), columns=list('ABCDEF'))
dft.head()

,A,B,C,D,E,F
0,0.739179,0.007291,0.251444,0.312501,0.420025,-0.555057
1,0.551888,-0.916458,1.267054,-0.155771,1.163977,1.009823
2,-0.408805,0.464986,1.036291,0.060136,1.056598,0.197868
3,0.723118,0.060825,1.855839,0.498021,0.726894,-1.530539
4,-1.489332,-0.151683,-0.547574,-0.769638,1.414344,-0.316152


In [118]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                  index=list('abcdef'),
                  columns=list('ABCD'))
df1

,A,B,C,D
a,0.329158,1.342746,0.543108,-0.205932
b,0.400815,0.825991,1.105481,0.052432
c,0.310453,-0.212980,-0.388222,0.577583
d,0.901756,1.150435,-0.435394,1.210392
e,-0.392809,0.787428,1.564472,1.246658
f,-1.168040,-0.872230,0.369906,0.742997


In [119]:
df1.loc[lambda df: df['A'] > 0, :]

,A,B,C,D
a,0.329158,1.342746,0.543108,-0.205932
b,0.400815,0.825991,1.105481,0.052432
c,0.310453,-0.212980,-0.388222,0.577583
d,0.901756,1.150435,-0.435394,1.210392


In [127]:
bb = pd.read_csv('data/baseball.csv', index_col='id')
bb.groupby(['year', 'team']).sum().loc[lambda df: df['r'] > 100]

stint    g    ab    r    h  X2b  X3b  hr    rbi    sb   cs   bb  \
year team                                                                    
2007 CIN       6  379   745  101  203   35    2  36  125.0  10.0  1.0  105   
     DET       5  301  1062  162  283   54    4  37  144.0  24.0  7.0   97   
     HOU       4  311   926  109  218   47    6  14   77.0  10.0  4.0   60   
     LAN      11  413  1021  153  293   61    3  36  154.0   7.0  5.0  114   
     NYN      13  622  1854  240  509  101    3  61  243.0  22.0  4.0  174   
     SFN       5  482  1305  198  337   67    6  40  171.0  26.0  7.0  235   
     TEX       2  198   729  115  200   40    4  28  115.0  21.0  4.0   73   
     TOR       4  459  1408  187  378   96    2  58  223.0   4.0  2.0  190   

              so   ibb   hbp    sh    sf  gidp  
year team                                       
2007 CIN   127.0  14.0   1.0   1.0  15.0  18.0  
     DET   176.0   3.0  10.0   4.0   8.0  28.0  
     HOU   212.0   3.0   9.0  16.0   6.0  17.0  
     LAN   141.0   8.0   9.0   3.0   8.0  29.0  
     NYN   310.0  24.0  23.0  18.0  15.0  48.0  
     SFN   188.0  51.0   8.0  16.0   6.0  41.0  
     TEX   140.0   4.0   5.0   2.0   8.0  16.0  
     TOR   265.0  16.0  12.0   4.0  16.0  38.0

In [8]:
dfd = pd.DataFrame({'A': [1, 2, 3],
                    'B': [4, 5, 6]},
                   index=list('abc'))
dfd.A[['a', 'c']]
dfd.loc[['a','c'],'A']
dfd.loc[dfd.index[[0,2]], 'A']

a    1
c    3
Name: A, dtype: int64

In [9]:
dfd.iloc[[0, 2], dfd.columns.get_indexer(['A', 'B'])]

,A,B
a,1,4
c,3,6


In [11]:
s=pd.Series([ 1, 2, 3, ])
labels = [ 1, 2, 3, ]
s.loc[s.index.intersection(labels)]

1    2
2    3
dtype: int64

In [18]:
s = pd.Series([0, 1, 2, 3, 4, 5])
s.sample(n=3)
s.sample(frac=0.3)

1    1
4    4
dtype: int64

In [20]:
s = pd.Series([0, 1, 2, 3, 4, 5])
s.sample(6, replace=False)
s.sample(n=6, replace=True)

0    0
2    2
5    5
5    5
5    5
4    4
dtype: int64

In [21]:
s = pd.Series([0, 1, 2, 3, 4, 5])
example_weights = [0, 0, 0.2, 0.2, 0.2, 0.4]
s.sample(n=3, weights=example_weights)

4    4
5    5
2    2
dtype: int64

In [24]:
df2 = pd.DataFrame({'col1': [9, 8, 7, 6], 'weight_column': [0.4, 0.3, 0.2, 0.1]})
df2.sample(3, weights='weight_column')

,col1,weight_column
1,8,0.3
2,7,0.2
0,9,0.4


In [36]:
dfi = pd.DataFrame(np.arange(6).reshape(3, 2), columns=['A', 'B'])

In [37]:
%timeit dfi.loc[:, 'C'] = dfi.loc[:, 'A']

272 µs ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
%timeit dfi['C'] = dfi['A']

99.5 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [42]:
s = pd.Series(range(-3, 4))
s[s > 0]
s[(s < -1) | (s > 0.5)]
s[~(s < 0)]

3    0
4    1
5    2
6    3
dtype: int64

In [53]:
df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                    'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                    'c': np.random.randn(7)})
df2
# only want 'two' or 'three'
criterion = df2['a'].map(lambda x: x.startswith('t'))
df2[criterion]
# equivalent but slower
criterion = [x.startswith('t') for x in df2.a]
df2[criterion]
# Multiple criteria
df2[criterion & (df2['b'] == 'x')]

,a,b,c
3,three,x,0.43522


In [50]:
%timeit df2[criterion]

270 µs ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [52]:
%timeit df2[criterion]

260 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [69]:
df = pd.DataFrame([[1, 2], [3, 4], [5, 6]],
                  index=list('abc'),
                  columns=['A', 'B'])
df
s = (df['A'] > 2)
df.loc[s, 'B']
#df.iloc[s,1] # ValueError: Location based indexing can only have
df.iloc[s.values, 1] # works perfectly
# s returns a series but s.values returns a numpyarray

array([False,  True,  True])

In [76]:
dft = DataFrame({
    'a':['a', 'b', 'c'],
    'b':['sorted', 'sorted', 'unsorted']
})
dft['b'] = dft['b'].str.title()
dft['b'] = dft['b'].astype('category')
dft.dtypes

a      object
b    category
dtype: object

In [90]:
s = pd.Series(np.arange(10,51,10), index=np.arange(5), dtype='int64')
s.index.isin([2, 4])
s[s.index.isin([2, 4, 6])]
s.reindex([2, 4, 6])

2    30.0
4    50.0
6     NaN
dtype: float64

In [95]:
l = np.array([1, 3, 2, 4])
s = 6
np.any(np.isin(l, (s-l)))

True

In [103]:
s_mi = pd.Series(np.arange(6),
        index=pd.MultiIndex.from_product([[0, 1], ['a', 'b', 'c']]))
s_mi.index.isin([(1, 'a'), (2, 'b'), (0, 'c')])
s_mi[s_mi.index.isin([(1, 'a'), (2, 'b'), (0, 'c')])]
                        
                                                                         

0  c    2
1  a    3
dtype: int64

In [118]:
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],
                   'ids2': ['a', 'n', 'c', 'n']})
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}
mask = df.isin(values).all(axis=1)
df[mask]

,vals,ids,ids2
0,1.0,a,a
1,NaN,b,NaN
2,3.0,NaN,c
3,NaN,NaN,NaN


In [121]:
s = pd.Series(range(-3, 4))
s[s>0]
s.where(s>0)

0    NaN
1    NaN
2    NaN
3    NaN
4    1.0
5    2.0
6    3.0
dtype: float64

In [126]:
dates=pd.date_range('1/2/2021', periods=8)
df = pd.DataFrame(np.random.randn(8, 4), columns=list("ABCD"),
                 index=dates)
df[df<0]
df.where(df<0, -df)

,A,B,C,D
2021-01-02,-0.431607,-0.302695,-0.587823,-0.696161
2021-01-03,-1.080251,-0.925235,-0.875016,-0.117676
2021-01-04,-0.264650,-0.211355,-0.197028,-1.652920
2021-01-05,-1.732677,-0.759164,-0.415342,-0.116503
2021-01-06,-0.642523,-0.313198,-1.314990,-0.783542
2021-01-07,-1.316334,-0.455331,-0.427571,-0.309025
2021-01-08,-0.541618,-0.016221,-1.658743,-1.712345
2021-01-09,-0.038927,-0.125983,-0.595635,-1.339990


In [129]:
s2 = s.copy()
s2[s2 < 0] = 0
s2

df2= df.copy()
df2[df2<0] = 0
df2

,A,B,C,D
2021-01-02,0.000000,0.000000,0.587823,0.000000
2021-01-03,0.000000,0.000000,0.000000,0.000000
2021-01-04,0.264650,0.211355,0.197028,0.000000
2021-01-05,1.732677,0.000000,0.415342,0.000000
2021-01-06,0.642523,0.313198,1.314990,0.783542
2021-01-07,0.000000,0.455331,0.000000,0.000000
2021-01-08,0.541618,0.000000,1.658743,0.000000
2021-01-09,0.000000,0.000000,0.595635,0.000000


In [132]:
df_orig = df.copy()
df_orig.where(df_orig>0, 0., inplace=True)
df_orig

,A,B,C,D
2021-01-02,0.000000,0.000000,0.587823,0.000000
2021-01-03,0.000000,0.000000,0.000000,0.000000
2021-01-04,0.264650,0.211355,0.197028,0.000000
2021-01-05,1.732677,0.000000,0.415342,0.000000
2021-01-06,0.642523,0.313198,1.314990,0.783542
2021-01-07,0.000000,0.455331,0.000000,0.000000
2021-01-08,0.541618,0.000000,1.658743,0.000000
2021-01-09,0.000000,0.000000,0.595635,0.000000


In [133]:
%timeit df.where(df < 0, -df) # 1.22 ms ± 113 µs per loop

1.22 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [134]:
%timeit np.where(df < 0, df, -df) # 276 µs ± 27.7 µs per loop

276 µs ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [141]:
df2 = df.copy()
print(df2)
df2[df2[1:4] > 0] = 3
df2

                   A         B         C         D
2021-01-02 -0.431607 -0.302695  0.587823 -0.696161
2021-01-03 -1.080251 -0.925235 -0.875016 -0.117676
2021-01-04  0.264650  0.211355  0.197028 -1.652920
2021-01-05  1.732677 -0.759164  0.415342 -0.116503
2021-01-06  0.642523  0.313198  1.314990  0.783542
2021-01-07 -1.316334  0.455331 -0.427571 -0.309025
2021-01-08  0.541618 -0.016221  1.658743 -1.712345
2021-01-09 -0.038927 -0.125983  0.595635 -1.339990


,A,B,C,D
2021-01-02,-0.431607,-0.302695,0.587823,-0.696161
2021-01-03,-1.080251,-0.925235,-0.875016,-0.117676
2021-01-04,3.000000,3.000000,3.000000,-1.652920
2021-01-05,3.000000,-0.759164,3.000000,-0.116503
2021-01-06,0.642523,0.313198,1.314990,0.783542
2021-01-07,-1.316334,0.455331,-0.427571,-0.309025
2021-01-08,0.541618,-0.016221,1.658743,-1.712345
2021-01-09,-0.038927,-0.125983,0.595635,-1.339990


In [168]:
df2 = DataFrame({
    'a':np.arange(-3,3),
    'b':np.arange(-2,4),
    'c':np.arange(-1,5)
})
df2


,a,b,c
0,-3,-2,-1
1,-2,-1,0
2,-1,0,1
3,0,1,2
4,1,2,3
5,2,3,4


In [171]:
%timeit df2.where(df2>0, df2['a'], axis=0) # 939 µs ± 31.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

939 µs ± 31.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [172]:
%timeit df2.apply(lambda x,y: x.where(x>0,y), y=df2['a']) # 2.99 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

2.99 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [186]:
%timeit np.where(df2>0, df2, df2[list(len(df2.columns)*'a')])

809 µs ± 29.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [185]:
len(df2.columns)

3

In [196]:
df3 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df3.where(lambda x:x>4, lambda x:x+10)
df3.where(df3>4, lambda x:x+10)

,A,B,C
0,11,14,7
1,12,5,8
2,13,6,9


In [200]:
print(s.where(s>0))
s.mask(s>0)

0    NaN
1    NaN
2    NaN
3    NaN
4    1.0
5    2.0
6    3.0
dtype: float64


0   -3.0
1   -2.0
2   -1.0
3    0.0
4    NaN
5    NaN
6    NaN
dtype: float64

In [203]:
df3.mask(df3>4, lambda x:x+10)

,A,B,C
0,1,4,17
1,2,15,18
2,3,16,19


In [204]:
df = pd.DataFrame({'col1': list('ABBC'), 'col2': list('ZZXY')})
df['color'] = np.where(df['col2']=='Z', 'green', 'red')
df

,col1,col2,color
0,A,Z,green
1,B,Z,green
2,B,X,red
3,C,Y,red


In [ ]:
conditions = [
    (df['col1']='A') & (df['col2']='Z'),
    (df['col1']='B') & (df['col2']='Z'),
    (df['col1']='B')
]

choices = ['yellow', 'blue', 'blue']

df['color'] = np.select(conditions, choices, default='black')